# PydMagic examples

## Install the ipython extension and dependencies

In [1]:
!curl -s https://raw.githubusercontent.com/Reavershark/PydMagic/2023-update-squash/pyd_magic.py > pyd_magic.py
!pip install mergedict pyd >/dev/null
!apt-get update >/dev/null && apt-get install -y ldc dub >/dev/null
%load_ext pyd_magic
!rm pyd_magic.py

## General type conversion demo

In [2]:
%%pyd

import std;

@pdef!() auto type_demo() {
    return tuple(
        "Hello world!",
        true,
        1,
        5.iota.map!"a*2".array,
        ["a": 1],
        (int a) => a + 1
    );
}

In [3]:
print(type_demo())
print(type_demo()[-1](1)) # Call the lambda (the last tuple element)

('Hello world!', True, 1, [0, 2, 4, 6, 8], {'a': 1}, <PydFunc object at 0x7f4c636910a0>)
2


 ## Using custom python classes from D

In [4]:
class SomeClass:
  a_field = 1
  def test(self) -> int:
    return 2

In [5]:
%%pyd

@pdef!() auto py_class_demo() {
    PydObject py_obj = py_eval("SomeClass()", "__main__");
    py_obj.method("test"); // return value is discarded
    py_obj.a_field = 10;
    return py_obj;
}

In [6]:
some_class = py_class_demo()
print(some_class)
print(some_class.test())
print(some_class.a_field)

2
10


## Use D ranges in python

In [7]:
%%pyd
import std.range;

struct FibonacciRange
{
    // States of the Fibonacci generator
    int a = 1, b = 1;

    // The fibonacci range never ends
    enum empty = false;

    // Peek at the first element
    int front() const @property
    {
        return a;
    }

    // Remove the first element
    void popFront()
    {
        auto t = a;
        a = b;
        b = t + b;
    }
}

static assert(isInputRange!FibonacciRange);
static assert(isInfinite!FibonacciRange);

@pdef!() auto range_demo() {
  // Can't return infinite ranges to python
  return FibonacciRange().take(20);
}

In [8]:
for el in range_demo():
  print(el, end=" ")

1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 2584 4181 6765 

## Catching python exceptions in D

In [9]:
%%pyd

@pdef!() string py_exception_demo(PydObject some_lambda) {
  bool caught_exception = false;
  try {
      some_lambda();
  } catch (PythonException e) {
      caught_exception = true;
  }
  return caught_exception ? "Caught a PythonException" : "Didn't catch anything";
}

In [10]:
def just_throw():
  raise Exception('error')

print(py_exception_demo(lambda: None))
print(py_exception_demo(lambda: just_throw()))

Didn't catch anything
Caught a PythonException


## Catching D exceptions in python

In [11]:
%%pyd

@pdef!() void d_exception_demo() {
    throw new Exception("message");
}

@pdef!() void print_first_n_lines(string msg, int n) {
  import std : split, take, join;
  auto print = py_eval("print");
  
  string shortened = msg.split("\n").take(n).join("\n");
  print(shortened);
}

In [12]:
# Rewritten in d in the previous cell
#def print_first_n_lines(msg: str, n: int) -> None:
#  print('\n'.join(msg.split("\n")[0:n]))

try:
  d_exception_demo()
except Exception as e:
  print(type(e))
  print_first_n_lines(str(e), 15)

<class 'RuntimeError'>
D Exception:
object.Exception@/root/.cache/ipython/pyd/_pyd_magic_0717fb6d456ecdb75f1a8e32f07192f4/_pyd_magic_0717fb6d456ecdb75f1a8e32f07192f4.d(14): message
----------------
??:? void _pyd_magic_0717fb6d456ecdb75f1a8e32f07192f4.d_exception_demo() [0x7f4c42ace936]
??:? void pyd.func_wrap.applyPyTupleToAlias!(_pyd_magic_0717fb6d456ecdb75f1a8e32f07192f4.d_exception_demo(), "d_exception_demo").applyPyTupleToAlias(deimos.python.object.PyObject*, deimos.python.object.PyObject*) [0x7f4c42ae491d]
??:? deimos.python.object.PyObject* pyd.func_wrap.pyApplyToAlias!(_pyd_magic_0717fb6d456ecdb75f1a8e32f07192f4.d_exception_demo(), "d_exception_demo").pyApplyToAlias(deimos.python.object.PyObject*, deimos.python.object.PyObject*) [0x7f4c42ae477c]
??:? deimos.python.object.PyObject* pyd.func_wrap.function_wrap!(_pyd_magic_0717fb6d456ecdb75f1a8e32f07192f4.d_exception_demo(), "d_exception_demo").func(deimos.python.object.PyObject*, deimos.python.object.PyObject*, deimos.python.obje

## Parsing formatted strings using std.format + benchmark

In [13]:
%%pyd --dub_args="--build=release-nobounds"

import std.typecons;
import std.format;

@pdef!() Tuple!(int, int) parse_point_d(string s)
{
    Tuple!(int, int) t;
    s.formattedRead!"{%d;%d}"(t[0], t[1]);
    return t;
}

@pdef!() Tuple!(int, int)[] parse_point_list_d(string[] arr)
{
    Tuple!(int, int)[] tuples;
    tuples.length = arr.length;
 
    foreach (i, s; arr)
        s.formattedRead!"{%d;%d}"(tuples[i][0], tuples[i][1]);

    return tuples;
}

In [14]:
def parse_point_py(s: str) -> tuple:
  assert len(s) >= 5
  s = s[1:-1]
  s_split = s.split(';')
  return (int(s_split[0]), int(s_split[1]))

def parse_point_list_py(l: list) -> list:
  return [parse_point_py(x) for x in l]

In [15]:
points = ["{0;1}", "{2;3}", "{-1;0}"]

print(parse_point_list_d(points))
print(parse_point_list_py(points))

from time import time

points *= 1000

start = time()
for i in range(10_000):
  [parse_point_d(point) for point in points]
end = time()
print("parse_point_d:", end - start)

start = time()
for i in range(10_000):
  [parse_point_py(point) for point in points]
end = time()
print("parse_point_py:", end - start)

start = time()
for i in range(10_000):
  parse_point_list_d(points) # marginally faster
end = time()
print("parse_point_list_d:", end - start)

start = time()
for i in range(10_000):
  parse_point_list_py(points)
end = time()
print("parse_point_list_py:", end - start)

[(0, 1), (2, 3), (-1, 0)]
[(0, 1), (2, 3), (-1, 0)]
parse_point_d: 24.900002479553223
parse_point_py: 21.717164516448975
parse_point_list_d: 16.407686948776245
parse_point_list_py: 21.314393758773804
